# QUBO Portfolio Optimisation for a Single Football Match

This notebook demonstrates a **portfolio optimisation engine for sportsbook bets** using a QUBO / Ising formulation and the ORBIT probabilistic simulator.

(**TODO**: @EhsanMassah worth mentioning the provision of a tool for calculating correlations over a time-series of odds?)

Our solution focuses on **optimising a portfolio of sportsbook bets according to the user’s risk appetite**, rather than searching for strict risk-free arbitrage.

- Each bet is treated as a **node** in a graph, with a weight equal to its **expected value (EV)** based on our probability estimate and the market odds.
- **Edges** represent **correlation or overlap between bets** – for example, multiple bets depending on the same match outcome or scoreline.
- The optimisation goal is to select a subset of bets that **maximises total positive alpha** while penalising portfolios that are **over-exposed to the same risks**.

In other words, we help a bettor (or trading desk) allocate their bankroll intelligently:  
**focus on value, avoid redundant risk, and respect a chosen risk appetite.**

Although we demonstrate the method on sports betting, the underlying optimisation framework is general.  
Treating positions as nodes, their relationships as edges, and minimising an energy function to pick a portfolio can be applied in other financial settings too – for example **options selection**, **trades within the same asset class**, or **small diversified portfolios** where interactions matter.

**TODO**: @EhsanMassah Similarly, the reverse problem, "given a desired expected value, minimise the risk", has applications in insurance...

---

## Structure of this notebook

1. **Data preparation**  
   - Load and clean a set of markets for a single football match.
   - In our case we are using a recent game between FC Barcelona and Atlético Madrid.
   - Build a time series of odds and implied probabilities for multiple markets.

2. **QUBO formulation (portfolio objective)**  
   - Compute the **expected value (EV)** of each bet.  
   - Build a **QUBO energy function** where:  
     - linear terms encode the value of including a discrete allocation of each bet;
     - quadratic terms encode correlation / overlap penalties between bets. Deviations between the sum of the correlations and the user's risk appetite penalises the energy function.

3. **TODO**: @Asd2812 **Parameter Coefficient Estimation**
   - **TODO**: Still to do, Orbit was crashing af for me so I couldn't run any proper simulations to get parameter coefficient estimation done.
   - Given the objective coefficient, calculate the optimal penalty coefficient.
   - Plot this against the efficient

3. **Brute-force validation (small N)**  
   - For a modest number of markets (e.g. N ≈ 20), exhaustively evaluate all 2^N portfolios.  
   - Use this to obtain the exact ground state and validate the quality of ORBIT’s solution.
   - Analysis of the brute-force algorithm reveals what we expected, a correct yet very slow answer.
   - In a high-pace environment with a growing number of betting markets, waiting minutes to get the outcome of a small number of markets is far from optimal.
   - **TODO** Check if this is the right place to post results or not.

4. **QUBO → Ising mapping and ORBIT optimisation**  
   - Map the QUBO to an Ising model with couplings J and fields h.  
   - Run the ORBIT simulator to approximately minimise the Ising energy.  
   - Decode the resulting spin configuration back into a set of bets.

5. **Comparison and interpretation**  
   - Multiple classical solutions exist, with two of the more famous ones being simulated annealing and greedy algorithms.
   - Both of these heuristic models have their pros and cons.
   - Greedy algorithms reach sub-second speeds yet never get fall in the top 15% of most optimised portfolios.
   - Simulated annealing find the optimal betting pattern at comparable speeds to that of probabilistic computing.
   - However it cannot handle the actual size of sports betting markets, as handling simply over 50 markets causes the algorithm to run extremely slowly.
   - **TODO**: Compare ORBIT’s solution to classical alternatives. We can use the results from the simulation done prior to tonight/this morning.
   - **TODO**: (continuation of the same TODO.) If so, then the results are similar in time and accuracy, but Orbit performs better at scale, hence the advantage.
   - Inspect the selected bets, the number of positions, and the implied “quality” of the portfolio. **TODO**: Plot on the efficient frontier. Decide if this is needed or not for the notebook code

The emphasis is not on speed or large-scale deployment in this notebook, but on a **clear, end-to-end demonstration** of how a realistic optimisation problem is encoded as a QUBO and solved using probabilistic computing.

In [1]:
import numpy as np
import pandas as pd
import itertools
import time
import orbit 
from pathlib import Path

In [2]:
ROOT = Path.cwd()
DATA = 'data'
FILE = 'odds_long.csv'
ALPHA = 1.0
BETA = 5.0
A = 10

In [ ]:
# # TODO: Confirm removal of this cell since the data is being loaded in the following cell
# csv_path = Path(f'{ROOT}/{DATA}/{FILE}')
# df = pd.read_csv(csv_path)

# required_cols = {'timestamp', 'market', 'odds', 'implied_prob'}
# missing = required_cols - set(df.columns)
# if missing:
#     raise ValueError(f"Missing required columns in CSV: {missing}")

# df['timestamp'] = pd.to_datetime(df['timestamp'])
# odds_long = df.copy()

# print("Loaded odds data from CSV:")
# odds_long.head()

Loaded odds data from CSV:


,timestamp,market,odds,implied_prob
0,2025-01-07 20:00:00,Market_01,3.080929,0.324577
1,2025-01-07 21:00:00,Market_01,3.110546,0.321487
2,2025-01-07 22:00:00,Market_01,3.136694,0.318807
3,2025-01-07 23:00:00,Market_01,3.233240,0.309287
4,2025-01-08 00:00:00,Market_01,3.177385,0.314724


## Loading in and preparing the data from the odds API

In [51]:
# Reading the csv data stored from the API
import pandas as pd
bar_v_atl_data = 'data/output-bar-v-atl-shortened.csv'
np.random.seed(42)

def load_and_prepare_data(file_path):
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Melt the dataframe: keep timestamp as id, convert all odds columns to rows
    df_long = df.melt(
        id_vars=['timestamp'],
        var_name='market',
        value_name='odds'
    )
    
    # Calculate implied probability: p = 1 / odds
    df_long['implied_prob'] = 1.0 / df_long['odds']
    df_long['predicted_prob'] = df_long['implied_prob'] + np.random.normal(0, 0.1, size=len(df_long))
    df_long['expected_value'] = df_long['predicted_prob'] * df_long['odds']
    
    # Calculate the correlation matrix of the implied probabilities
    corr_df = df_long.pivot_table(
    index='timestamp',
    columns='market',
    values='implied_prob'
    ).corr()
    
    return df_long, corr_df

df, bar_v_atl_corr = load_and_prepare_data(bar_v_atl_data)
df.head()


,timestamp,market,odds,implied_prob,predicted_prob,expected_value
0,2025-11-23 16:34:21,money_line_home,1.740,0.574713,0.624384,1.086428
1,2025-11-23 16:35:21,money_line_home,1.746,0.572738,0.558911,0.975859
2,2025-11-23 16:38:01,money_line_home,1.769,0.565291,0.630060,1.114576
3,2025-11-23 16:38:31,money_line_home,1.793,0.557724,0.710027,1.273079
4,2025-11-23 16:44:46,money_line_home,1.781,0.561482,0.538067,0.958297


## QUBO formulation: betting portfolio as a binary optimisation problem

We model the selection of bets as a **binary vector**:

- Let $x_i \in \set{0,1}$ indicate whether bet $i$ is included in the portfolio.  
- For each bet, we define an **expected value per unit stake**:
  $$\text{EV}_i = p_i \cdot \text{odds}_i$$
  where $p_i$ is our (subjective or model-based) probability for that outcome and $\text{odds}_i$ is the decimal odds.

We model the risk appetite as a maximum volatility, $\sigma_{\max}$. The true portfolio variance is modelled by
$$\sigma^2(\mathbf{x}) = \mathbf{x}^\top \Sigma \mathbf{x} = \sum_{i,j} \Sigma_{i,j}x_i x_j$$

To enforce allocation restraints, we ensure $\sum_i x_i \le A$ via a penalty term:
$$(A - \sum_i x_i)^2 = A^2 - 2A \sum_i x_i + \sum_{i,j}x_i x_j$$

Combining the expected value objective, the risk penalty, and the allocation restraint, we obtain:
$$\begin{align*}
E_{\text{QUBO}}(x) &= - \alpha \sum\limits_{i} \mu_{i} x_{i} + \beta \sum\limits_{i,j} \Sigma_{i,j}x_{i} x_{j} + A^{2} - 2A \sum\limits_{i} x_{i} + \sum\limits_{i,j}x_{i} x_{j} \\
&= - \sum\limits_{i}(\alpha \mu_{i} + 2A)x_{i} + \sum\limits_{i,j}(\beta \Sigma_{i, j} + 1)x_{i}x_{j} + A^{2}
\end{align*}$$

We then construct a **QUBO energy function** of the form:
$$E_{\text{QUBO}}(x) = - \sum_i \mu_i x_i + \sum_{i<j} Q_{ij} x_i x_j + \text{const}$$

The **linear term** $\mu_i$ is taken as
$$\mu_i = \alpha \text{EV}_i + 2A$$
so that bets with higher expected value contribute **lower energy** when selected (since we minimise $E$).

The **quadratic term** $Q_{ij}$ encodes **correlations or overlaps** between bets:
  - If two bets load on the same underlying match or outcome, a positive $Q_{ij}$ penalises taking them together.
  - If two bets are relatively independent, $Q_{ij}$ will be small.

The linear and quadratic terms both include elements of the allocation restraint to penalise exceeding $A$, too.

The overall effect is:

- Portfolios with **many high-EV bets** have **low linear energy**.  
- Portfolios that are **over-exposed to the same outcome** incur **quadratic penalties**.  
- Portfolios are penalised for exceeding the allocation constraint.

By minimising $E_{\text{QUBO}}(x)$ according to various $\alpha, \beta$, we search for a portfolio that balances **value (alpha)** and **risk concentration**, consistent with the user’s risk appetite and allocation size.


In [52]:
def expected_value(p, odds):
    """
    Expected profit per unit stake:
    EV = p * odds
    """
    return p * odds

def portfolio_energy_qubo(x, mu, cov, alpha, beta, A):
    """
    QUBO energy:
    x : 1D numpy array of 0/1 of length N
    mu : 1D numpy array of expected values of length N
    cov : 2D numpy array (NxN) covariance matrix
    alpha : float, weight for linear term
    beta : float, weight for quadratic penalty
    A : float, target allocation (max allocation size)
    """
    linear = - np.sum((alpha * mu + 2.0 * A) * x)
    Q = beta * cov + 1.0
    quadratic = x @ Q @ x
    return linear + quadratic + A**2

In [53]:
def compute_market_correlation(df: pd.DataFrame, value_col="implied_prob"):
    """
    Compute the correlation matrix of N betting markets based on
    implied probabilities (or odds).

    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns ['timestamp', 'market', value_col]
    value_col : str
        Column name to compute correlation on ("implied_prob" or "odds")

    Returns
    -------
    corr_matrix : pd.DataFrame
        N×N correlation matrix of markets.
    wide_df : pd.DataFrame
        Time-indexed wide DataFrame used for correlation.
    """

    wide_df = df.pivot_table(
        index="timestamp",
        columns="market",
        values=value_col
    )
    cov = wide_df.corr()
    return cov, wide_df

In [54]:
#Taking the lastest odds from the API
final_snapshot = (
    df.sort_values("timestamp")
      .groupby("market")
      .tail(1)
      .set_index("market")
)
final_snapshot

,timestamp,odds,implied_prob,predicted_prob,expected_value
market,,,,,
spread_-0.75_away,2025-12-02 14:59:30,2.030,0.492611,0.629896,1.278688
spread_-1.25_away,2025-12-02 14:59:30,1.595,0.626959,0.607593,0.969111
spread_-1.0_home,2025-12-02 14:59:30,2.150,0.465116,0.421954,0.907202
spread_-0.75_home,2025-12-02 14:59:30,1.884,0.530786,0.511403,0.963483
spread_-0.5_home,2025-12-02 14:59:30,1.694,0.590319,0.652019,1.104521
spread_-1.25_home,2025-12-02 14:59:30,2.450,0.408163,0.527228,1.291708
totals_4.5_over,2025-12-02 14:59:30,3.330,0.300300,0.268792,0.895076
spread_-1.5_away,2025-12-02 14:59:30,1.487,0.672495,0.644427,0.958264
spread_-0.25_home,2025-12-02 14:59:30,1.512,0.661376,0.507818,0.767821


In [55]:
# Prepare variables for calculations
mu = final_snapshot["expected_value"].values
cov, _ = compute_market_correlation(df)
cov

market,money_line_away,money_line_draw,money_line_home,spread_-0.25_away,spread_-0.25_home,spread_-0.5_away,spread_-0.5_home,spread_-0.75_away,spread_-0.75_home,spread_-1.0_away,...,spread_-1.5_away,spread_-1.5_home,spread_-1.75_away,spread_-1.75_home,spread_0.0_away,spread_0.0_home,spread_0.25_away,spread_0.25_home,totals_4.5_over,totals_4.5_under
market,,,,,,,,,,,,,,,,,,,,,
money_line_away,1.000000,0.679587,-0.900308,0.978194,-0.913947,0.972703,-0.896765,0.972731,-0.907498,0.966216,...,0.938733,-0.919896,0.900841,-0.926185,0.990212,-0.920043,0.991884,-0.910065,0.197777,0.171263
money_line_draw,0.679587,1.000000,-0.656940,0.789322,-0.604573,0.808371,-0.678509,0.814077,-0.681312,0.815460,...,0.855909,-0.693628,0.888270,-0.681772,0.728659,-0.522398,0.712339,-0.457628,-0.287524,0.417499
money_line_home,-0.900308,-0.656940,1.000000,-0.932326,0.985406,-0.923486,0.992860,-0.915507,0.991780,-0.905421,...,-0.857074,0.964166,-0.820250,0.953211,-0.927893,0.967879,-0.920500,0.940308,-0.178507,-0.097829
spread_-0.25_away,0.978194,0.789322,-0.932326,1.000000,-0.926574,0.996632,-0.935387,0.994303,-0.943482,0.988913,...,0.970930,-0.943636,0.947950,-0.941288,0.992284,-0.909078,0.990265,-0.881093,0.111964,0.211943
spread_-0.25_home,-0.913947,-0.604573,0.985406,-0.926574,1.000000,-0.917990,0.986090,-0.911216,0.985924,-0.900528,...,-0.845004,0.969746,-0.796782,0.962704,-0.935866,0.989413,-0.924535,0.974237,-0.204623,-0.085567
spread_-0.5_away,0.972703,0.808371,-0.923486,0.996632,-0.917990,1.000000,-0.927364,0.995460,-0.936631,0.991942,...,0.976907,-0.942283,0.954737,-0.940423,0.987680,-0.896378,0.983438,-0.868170,0.085434,0.230288
spread_-0.5_home,-0.896765,-0.678509,0.992860,-0.935387,0.986090,-0.927364,1.000000,-0.920997,0.992358,-0.907923,...,-0.862218,0.965869,-0.828178,0.952718,-0.928319,0.964620,-0.920967,0.933469,-0.155452,-0.112882
spread_-0.75_away,0.972731,0.814077,-0.915507,0.994303,-0.911216,0.995460,-0.920997,1.000000,-0.928520,0.993587,...,0.981680,-0.942871,0.959566,-0.943319,0.985894,-0.889801,0.980005,-0.863234,0.063231,0.260093
spread_-0.75_home,-0.907498,-0.681312,0.991780,-0.943482,0.985924,-0.936631,0.992358,-0.928520,1.000000,-0.922258,...,-0.878071,0.975781,-0.843250,0.964892,-0.937493,0.966474,-0.929178,0.937791,-0.143533,-0.134650


In [56]:
#Brute-Force Calculations
N = len(final_snapshot)
all_results = []

start_time = time.perf_counter() 

for bits in itertools.product([0, 1], repeat=N):
    x = np.array(bits)
    E = portfolio_energy_qubo(x, mu, cov.values, alpha=ALPHA, beta=BETA, A=A)
    all_results.append({
        "x": bits,
        "E": E,
        "num_bets": x.sum()
    })

end_time = time.perf_counter() 
elapsed_bf = end_time - start_time

results_df = (
    pd.DataFrame(all_results)
      .sort_values("E", ascending=True)
      .reset_index(drop=True)
)

print(results_df.head(32))
print(f"\nChecked {2**N} Configurations for N={N}.")
print(f"Brute Force - Time to Ground State: {elapsed_bf:.4f} seconds")

                                                    x         E  num_bets
0   (0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, ... -1.136825        10
1   (0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, ... -1.027171        10
2   (0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, ... -0.916320        10
3   (0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, ... -0.915095        10
4   (1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, ... -0.897673        10
5   (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ... -0.884790        10
6   (0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, ... -0.853886        10
7   (0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, ... -0.853005        10
8   (0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, ... -0.836074        10
9   (0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, ... -0.816149        10
10  (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, ... -0.804333        10
11  (0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, ... -0.775156        10
12  (0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 

## QUBO → Ising mapping and interpretation of scaling factors

The ORBIT simulator expects an **Ising model** of the form
$$E_{\text{Ising}}(s) = \sum_i h_i s_i + \sum_{i<j} J_{ij} s_i s_j$$
where each spin variable $s_i \{-1, +1\}$.

Our QUBO is defined in terms of binary variables $x_i \in {0,1}$:
$$
E_{\text{QUBO}}(x) = - \sum\limits_{i}(\alpha \mu_{i} + 2A)x_{i} + \sum\limits_{i,j}(\beta \Sigma_{i, j} + 1)x_{i}x_{J} + A^{2}
$$

Note
$$
\sum_{i,j} (\beta \Sigma_{ij} + 1)x_i x_j = \sum_i (\beta \Sigma_{ii} + 1)x_i^2 + \sum_{i<j} 2(\beta \Sigma_{ij} + 1)x_i x_j
$$

Thus,
$$
E_{\text{QUBO}}(x) = \sum_i a_i x_i + \sum_{i<j} b_{ij} x_i x_j + \text{const} \\
a_i = -(\alpha \mu_i + 2A) + (\beta \Sigma_{ii} + 1) \\
b_{ij} = 2(\beta \Sigma_{ij} + 1), \quad i<j
$$

The standard mapping between these two formulations uses
$$x_i = \frac{1 + s_i}{2}$$

If we substitute $x_i = (1+s_i)/2$ into the QUBO and collect terms, we obtain an Ising model with:

- couplings
  $$J_{ij} = \frac{b_{ij}}{4}$$
- local fields
  $$h_i = \frac{a_i}{2} + \frac{1}{4}\sum_{j \neq i} b_{ij}$$

Thus,

$$
J_{ij} = \frac{1}{4} b_{ij} = \frac{1}{4}\, 2(\beta \Sigma_{ij} + 1) = \frac{1}{2}(\beta \Sigma_{ij} + 1), \quad i<j \\
J_{ij} = \frac{1}{2}(\beta \Sigma_{ij} + 1),\quad i<j
$$

and,
$$
a_i = -(\alpha \mu_i + 2A) + (\beta \Sigma_{ii} + 1) = -\alpha \mu_i - 2A + \beta \Sigma_{ii} + 1 \\
\sum_{j\neq i} b_{ij} = \sum_{j\neq i} 2(\beta \Sigma_{ij} + 1) = 2\sum_{j\neq i} (\beta \Sigma_{ij} + 1) \\
$$

giving,
$$
\begin{align*}
h_{i} &= \frac{a_i}{2} + \frac{1}{4}\sum_{j\neq i} b_{ij} \\
&= \frac{1}{2}(-\alpha \mu_i - 2A + \beta \Sigma_{ii} + 1) + \frac{1}{4}\cdot 2\sum_{j\neq i} (\beta \Sigma_{ij} + 1) \\
&= -\frac{\alpha}{2}\mu_i - A + \frac{1}{2}(\beta \Sigma_{ii} + 1) + \frac{1}{2}\sum_{j\neq i} (\beta \Sigma_{ij} + 1) \\
&= -\frac{\alpha}{2} \mu_{i} - A + \sum\limits_{ij} \frac{1}{2} (\beta \Sigma _{ij} + 1)\\
&= - \frac{\alpha}{2} \mu_{i} - A + \sum\limits_{ij} J_{ij}
\end{align*}
$$

Two important points for interpretation:

1. **Absolute energies differ, minimisers do not**  
   The Ising energy and the original QUBO energy can have different absolute values because of constant shifts and rescaling. What matters is that they share the **same minimising configuration**.

2. **Validation via brute force**  
   For the small problem size in this notebook, we perform an exhaustive search over all $2^N$ portfolios to find the exact ground state of the QUBO.  
   We then run ORBIT on the corresponding Ising instance and **compare ORBIT’s solution to the brute-force optimum**, confirming that the mapping and implementation are consistent.


In [62]:
np.sum(J, axis=0)

array([15.40293201, 18.17788857, 11.09246578, 15.57022967, 11.43538445,
       15.77798264, 10.97485239, 15.92926216, 10.76075032, 16.04892516,
        9.99796002, 16.70731725, 10.11215729, 17.0299786 , 10.15683599,
       17.6125103 , 10.00427114, 15.22979652, 11.74610993, 15.37797931,
       12.05982058,  8.92275713, 15.75036794])

In [ ]:
#Orbit Calculations
N = len(final_snapshot)
bets = final_snapshot.index.tolist()  

# Standard QUBO -> Ising mapping 
J = 0.5 * (BETA * cov.values + 1.0)
h = np.sum(J, axis=1) - 0.5 * ALPHA * mu - A # type: ignore

# J = 2 * (BETA * cov.values + 1.0)
# h = BETA * np.diag(cov.values) + 1.0 - ALPHA * mu - 2 * A # type: ignore

start_time = time.perf_counter()
result = orbit.optimize_ising(
    J,
    h,
    n_replicas=20,
    full_sweeps=200_000,
    beta_initial=0.35,
    beta_end=3.5,
    beta_step_interval=1,
)

elapsed_orb = time.perf_counter() - start_time

x_star = np.array(result.min_state)        

E_orbit = portfolio_energy_qubo(x_star, mu, cov.values, alpha=ALPHA, beta=BETA, A=A)

chosen_bets = [b for b, bit in zip(bets, x_star) if bit == 1]

print("=== ORBIT optimisation result ===")
print(f"Time to (approximately) find ground state: {elapsed_orb:.4f} seconds\n")

print("Bits  (x*):", x_star.tolist())
print("Number of bets in portfolio:", int(x_star.sum()))
print("Selected bets:", chosen_bets)

print("\nORBIT reported min_cost:", result.min_cost)
print("Objective E = portfolio_energy_qubo(x*, w, Q):", E_orbit)

# Optional: compare with brute-force ground state if you still have results_df
if 'results_df' in globals():
    print("\n=== Comparison with brute force ===")
    print("Brute-force best E:", results_df.loc[0, 'E'])
    print("Brute-force best x:", results_df.loc[0, 'x'])

[2025-12-07 13:31:03] INFO - orbit.simulator: Simulation starting...
[2025-12-07 13:38:30] INFO - orbit.simulator: Simulation completed in 420.14 seconds
=== ORBIT optimisation result ===
Time to (approximately) find ground state: 934.2726 seconds

Bits  (x*): [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]
Number of bets in portfolio: 10
Selected bets: ['spread_-1.25_away', 'spread_-0.5_home', 'totals_4.5_over', 'spread_-1.75_home', 'spread_0.0_away', 'money_line_away', 'spread_0.25_home', 'spread_0.25_away', 'spread_-0.5_away', 'totals_4.5_under']

ORBIT reported min_cost: -4.196188087535148
Objective E = portfolio_energy_qubo(x*, w, Q): 5.490855851806458

=== Comparison with brute force ===
Brute-force best E: -1.1368248567771815
Brute-force best x: (0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1)


In [66]:
# --- Check ORBIT solution against brute-force results and rank it ---
orbit_bits = tuple(int(b) for b in x_star)
matches = results_df[results_df["x"] == orbit_bits]

if matches.empty:
    print("⚠ ORBIT bitstring not found in brute-force results (this should not happen if N matches).")
else:
    match_idx = matches.index[0]   # 0-based index in results_df
    rank = match_idx + 1           # human-friendly rank (1 = best)

    E_best = results_df.loc[0, "E"]
    x_best = results_df.loc[0, "x"]
    E_orbit_bruteforce = matches.iloc[0]["E"]

    print("\n=== ORBIT vs Brute Force ===")
    print(f"ORBIT bitstring: {orbit_bits}")
    print(f"ORBIT energy from brute-force table: {E_orbit_bruteforce:.6f}")
    print(f"Ground state energy (brute force):   {E_best:.6f}")
    print(f"Ground state bitstring:              {x_best}")

    print(f"\nRank of ORBIT state among all 2^{N} configs: {rank} (1 = ground state)")
    print(f"Energy gap to ground state: {E_orbit_bruteforce - E_best:.6f}")

    # Optional: how many configs are strictly better / equal
    n_better = (results_df["E"] < E_orbit_bruteforce).sum()
    n_equal  = (results_df["E"] == E_orbit_bruteforce).sum()
    print("")
    print(f"Time to the lowest state using orbit: {elapsed_orb:.4f} seconds\n")
    print(f"Time to find the ground state using brute-force  : {elapsed_bf:.4f} seconds\n")


=== ORBIT vs Brute Force ===
ORBIT bitstring: (0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1)
ORBIT energy from brute-force table: 5.490856
Ground state energy (brute force):   -1.136825
Ground state bitstring:              (0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1)

Rank of ORBIT state among all 2^23 configs: 139440 (1 = ground state)
Energy gap to ground state: 6.627681

Time to the lowest state using orbit: 934.2726 seconds

Time to find the ground state using brute-force  : 71.1666 seconds



In [122]:
def sa_optimize_qubo(w, cov, n_steps=100_000, T_start=5.0, T_end=0.01, seed=123):
    rng = np.random.default_rng(seed)
    N = len(w)

    def energy(x):
        return portfolio_energy_qubo(x, w, cov, alpha=ALPHA, beta=BETA, A=A)  # same sign convention as brute-force

    x = rng.integers(0, 2, size=N, dtype=int)
    E = energy(x)

    best_x = x.copy()
    best_E = E

    for step in range(n_steps):
        T = T_start * (T_end / T_start) ** (step / max(1, n_steps - 1))

        i = rng.integers(0, N)
        x_new = x.copy()
        x_new[i] = 1 - x_new[i]

        E_new = energy(x_new)
        dE = E_new - E

        if dE <= 0 or rng.random() < np.exp(-dE / T):
            x, E = x_new, E_new
            if E < best_E:
                best_E = E
                best_x = x.copy()

    return best_x, best_E

start = time.perf_counter()
x_sa, E_sa = sa_optimize_qubo(mu, cov, n_steps=100_000)
elapsed_sa = time.perf_counter() - start

print("=== Simulated Annealing result ===")
print("Bits (x*):", x_sa.tolist())
print("Number of bets in portfolio:", int(x_sa.sum()))
print(f"SA energy: {E_sa:.6f}")
print(f"Time to (approximately) find ground state: {elapsed_sa:.4f} seconds")

if "results_df" in globals():
    sa_bits = tuple(int(b) for b in x_sa)
    matches = results_df[results_df["x"] == sa_bits]

    if matches.empty:
        print("\nSA bitstring not found in brute-force table.")
    else:
        idx = matches.index[0]
        rank = idx + 1
        E_best = results_df.loc[0, "E"]
        print("\n=== SA vs Brute Force ===")
        print(f"Rank of SA state among all 2^N configs: {rank} (1 = ground state)")
        print(f"Ground state energy (brute force): {E_best:.6f}")
        print(f"Energy gap to ground state: {E_sa - E_best:.6f}")

=== Simulated Annealing result ===
Bits (x*): [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1]
Number of bets in portfolio: 10
SA energy: -0.266829
Time to (approximately) find ground state: 10.1588 seconds

=== SA vs Brute Force ===
Rank of SA state among all 2^N configs: 6 (1 = ground state)
Ground state energy (brute force): -0.504562
Energy gap to ground state: 0.237733


In [123]:
# Greedy Optimisation Algorithm
# Based on: Andrew Vince. A framework for the greedy algorithm.
# Discrete Applied Mathematics, 121(1-3):247–260, 2002.

def greedy_optimize_qubo(mu, cov, alpha=ALPHA, beta=BETA, A=A):
    """
    Greedy algorithm for QUBO optimization following Vince (2002) framework.
    
    The algorithm starts with an empty solution and iteratively adds/removes
    variables (bits) that give the best local improvement in energy.
    
    Args:
        mu: 1D numpy array of expected values (linear coefficients)
        cov: 2D numpy array (NxN) covariance matrix (quadratic coefficients)
        alpha: float, weight for linear term
        beta: float, weight for quadratic penalty
        A: float, target allocation (max allocation size)
    
    Returns:
        x: 1D numpy array of 0/1 solution
        E: float, final energy value
    """
    N = len(mu)
    
    def energy(x):
        """Compute QUBO energy for given bitstring x."""
        return portfolio_energy_qubo(x, mu, cov, alpha=alpha, beta=beta, A=A)
    
    # Initialize: start with all zeros (empty portfolio)
    x = np.zeros(N, dtype=int)
    E = energy(x)
    
    improved = True
    iterations = 0
    max_iterations = N * 100  # Prevent infinite loops
    
    while improved and iterations < max_iterations:
        improved = False
        best_delta = 0
        best_idx = None
        
        # Try flipping each bit and find the best improvement
        for i in range(N):
            # Flip bit i
            x_new = x.copy()
            x_new[i] = 1 - x_new[i]
            
            # Compute energy difference efficiently
            # For QUBO: E = -sum((alpha*mu_i + 2A)*x_i) + sum((beta*cov_ij + 1)*x_i*x_j) + A^2
            # When flipping bit i from x_i to (1-x_i), the change is:
            # delta = E_new - E_old
            
            # Linear term change: -(alpha*mu_i + 2A) * (1 - 2*x_i)
            linear_delta = -(alpha * mu[i] + 2 * A) * (1 - 2 * x[i])
            
            # Quadratic term change: interactions with all other bits
            # When x_i flips, all terms involving x_i change
            quadratic_delta = 0
            for j in range(N):
                if j != i:
                    Q_ij = beta * cov[i, j] + 1
                    # Old contribution: Q_ij * x[i] * x[j]
                    # New contribution: Q_ij * (1-x[i]) * x[j]
                    # Change: Q_ij * (1 - 2*x[i]) * x[j]
                    quadratic_delta += Q_ij * (1 - 2 * x[i]) * x[j]
            
            # Self-interaction term (x_i^2 = x_i for binary variables)
            Q_ii = beta * cov[i, i] + 1
            if x[i] == 0:  # Adding: 0 -> 1, adds Q_ii
                self_delta = Q_ii
            else:  # Removing: 1 -> 0, subtracts Q_ii
                self_delta = -Q_ii
            
            delta = linear_delta + quadratic_delta + self_delta
            
            # Negative delta means energy decreases (improvement)
            if delta < best_delta:
                best_delta = delta
                best_idx = i
                improved = True
        
        # Apply the best flip if it improves energy
        if improved and best_idx is not None:
            x[best_idx] = 1 - x[best_idx]
            E = energy(x)  # Recompute for accuracy
            iterations += 1
        else:
            break
    
    return x, E

# Run greedy algorithm
start = time.perf_counter()
x_greedy, E_greedy = greedy_optimize_qubo(mu, cov.values)
elapsed_greedy = time.perf_counter() - start

print("=== Greedy Algorithm result ===")
print("Bits (x*):", x_greedy.tolist())
print("Number of bets in portfolio:", int(x_greedy.sum()))
print(f"Greedy energy: {E_greedy:.6f}")
print(f"Time to find local optimum: {elapsed_greedy:.4f} seconds")

# Compare with brute force if available
if "results_df" in globals():
    greedy_bits = tuple(int(b) for b in x_greedy)
    matches = results_df[results_df["x"] == greedy_bits]
    
    if matches.empty:
        print("\nGreedy bitstring not found in brute-force table.")
        # Find closest energy in brute force results
        energy_diff = np.abs(results_df["E"] - E_greedy)
        closest_idx = energy_diff.idxmin()
        closest_rank = closest_idx + 1
        E_best = results_df.loc[0, "E"]
        print(f"Closest brute-force energy: {results_df.loc[closest_idx, 'E']:.6f} (rank {closest_rank})")
        print(f"Energy gap to ground state: {E_greedy - E_best:.6f}")
    else:
        idx = matches.index[0]
        rank = idx + 1
        E_best = results_df.loc[0, "E"]
        print("\n=== Greedy vs Brute Force ===")
        print(f"Rank of Greedy state among all 2^N configs: {rank} (1 = ground state)")
        print(f"Ground state energy (brute force): {E_best:.6f}")
        print(f"Energy gap to ground state: {E_greedy - E_best:.6f}")

# Compare with Simulated Annealing
if "x_sa" in globals() and "E_sa" in globals():
    print("\n=== Greedy vs Simulated Annealing ===")
    print(f"SA energy: {E_sa:.6f}")
    print(f"Greedy energy: {E_greedy:.6f}")
    print(f"Energy difference (Greedy - SA): {E_greedy - E_sa:.6f}")
    print(f"SA time: {elapsed_sa:.4f} seconds")
    print(f"Greedy time: {elapsed_greedy:.4f} seconds")
    print(f"Speedup: {elapsed_sa / elapsed_greedy:.2f}x")
    
    # Check if solutions are the same
    if np.array_equal(x_greedy, x_sa):
        print("Solutions are identical!")
    else:
        print(f"Solutions differ in {np.sum(x_greedy != x_sa)} bits")

=== Greedy Algorithm result ===
Bits (x*): [1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1]
Number of bets in portfolio: 16
Greedy energy: 49.071229
Time to find local optimum: 0.0053 seconds

=== Greedy vs Brute Force ===
Rank of Greedy state among all 2^N configs: 5193560 (1 = ground state)
Ground state energy (brute force): -0.504562
Energy gap to ground state: 49.575791

=== Greedy vs Simulated Annealing ===
SA energy: -0.266829
Greedy energy: 49.071229
Energy difference (Greedy - SA): 49.338058
SA time: 10.1588 seconds
Greedy time: 0.0053 seconds
Speedup: 1904.98x
Solutions differ in 8 bits


## Summary and typical usage

In this notebook we have:

- Built a realistic **sportsbook portfolio optimisation** instance for a single football match.  
- Encoded the problem as a **QUBO** where:
  - linear terms reward high expected value bets,  
  - quadratic terms penalise correlated / overlapping exposures.  
- Exhaustively searched all \(2^N\) portfolios for a modest-sized universe (N ≈ 20) to obtain the **exact ground state**.
- Mapped the QUBO to an **Ising model** and used **ORBIT** to approximately minimise the Ising energy.
- Compared the ORBIT solution to the brute-force optimum and inspected the selected portfolio of bets.

For a typical user with more compute, the same pipeline can be applied to **larger sets of markets**:

- The data feed would come from live sportsbook APIs rather than static files.  
- The correlation structure \(Q\) can incorporate more sophisticated measures of dependence between markets.  
- ORBIT can be used to explore the energy landscape and propose **high-quality portfolios** within the user’s risk and bankroll constraints.

This notebook is therefore intended as an **end-to-end, validated proof of concept** that a real combinatorial portfolio problem in sports betting can be:

1. Formulated as a QUBO,
2. Mapped to an Ising model, and
3. Solved using probabilistic computing techniques such as ORBIT.
